# Chapter 10: Data Aggregation and Group Operations

In [1]:
import numpy as np
import pandas as pd

## 10.1 GroupBy Mechanics

**Split-apply-combine** for describing group operations:

1. Split - Data is split into groups based on one or more keys that you provide, performed an a particular axis.
2. Apply - A function is applied to each group, producing a new value.
3. Combine - Resultas of all functions are combined into a result object.

In [2]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                    'key2': ['one', 'two', 'one', 'two', 'one'],
                    'data1': np.random.randn(5),
                    'data2': np.random.rand(5)})
df

,key1,key2,data1,data2
0,a,one,0.145615,0.351670
1,a,two,-0.547849,0.388579
2,b,one,-0.913499,0.417067
3,b,two,0.604420,0.825327
4,a,one,2.185340,0.416138


Suppose you wanted to compute the mean of the `data1` column using the labels from `key1`.

Option 1: Access `data1` and call `groupby` with the column at `key1`.

In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [4]:
grouped.mean()

key1
a    0.594369
b   -0.154539
Name: data1, dtype: float64

Option 2: Instead pass multiple arrays as a list to get something different.

In [6]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     1.165478
      two    -0.547849
b     one    -0.913499
      two     0.604420
Name: data1, dtype: float64

In [7]:
means.unstack()

key2,one,two
key1,,
a,1.165478,-0.547849
b,-0.913499,0.604420


In thise example, the group keys are all Series.

In [8]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

df['data1'].groupby([states, years]).mean()

California  2005   -0.547849
            2006   -0.913499
Ohio        2005    0.375018
            2006    2.185340
Name: data1, dtype: float64

Often times the grouping information is found in the same DataFrame so just pass column names as the group keys.

In [9]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.594369,0.385462
b,-0.154539,0.621197


> Note: `df['key2']` is not numeric data, a *nuisance column*, which is excluded from the result.

In [10]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   1.165478  0.383904
     two  -0.547849  0.388579
b    one  -0.913499  0.417067
     two   0.604420  0.825327

A generally useful GroupBy method is `size`, which returns a Series containing group sizes.

In [11]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 10.1.1 Iterating Over Groups

The GroupBy object supports iteration, generating a sequence of 2-tuples.

In [12]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  0.145615  0.351670
1    a  two -0.547849  0.388579
4    a  one  2.185340  0.416138
b
  key1 key2     data1     data2
2    b  one -0.913499  0.417067
3    b  two  0.604420  0.825327


In [13]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.145615  0.351670
4    a  one  2.185340  0.416138
('a', 'two')
  key1 key2     data1     data2
1    a  two -0.547849  0.388579
('b', 'one')
  key1 key2     data1     data2
2    b  one -0.913499  0.417067
('b', 'two')
  key1 key2    data1     data2
3    b  two  0.60442  0.825327


A useful recipe is computing a dict of the data pieces as a one-liner.

In [14]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,-0.913499,0.417067
3,b,two,0.604420,0.825327


By default, `groupby` groups on `axis=0`, but can group on any axis.

In [15]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [17]:
grouped = df.groupby(df.dtypes, axis=1)

for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  0.145615  0.351670
1 -0.547849  0.388579
2 -0.913499  0.417067
3  0.604420  0.825327
4  2.185340  0.416138
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 10.1.2 Selecting a Column or Subset of Columns

Indexing a GroupBy object created from a DataFrame with a column name or array of column names has the effect of column subsetting for aggregation.

    df.groupby('key1')['data1']
    df.groupby('key1')[['data2']]

is the same as:

    df['data1'].groupby(df['key1'])
    df[['data2']].groupby(df['key1'])

As an example, to compute means for just `data2` column.

In [18]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.383904
     two   0.388579
b    one   0.417067
     two   0.825327

In [19]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [20]:
s_grouped.mean()

key1  key2
a     one     0.383904
      two     0.388579
b     one     0.417067
      two     0.825327
Name: data2, dtype: float64

### 10.1.3 Grouping with Dicts and Series

In [21]:
people = pd.DataFrame(np.random.randn(5, 5),
                        columns=['a', 'b', 'c', 'd', 'e'],
                        index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,0.660532,0.290632,-1.237698,-1.030799,-1.225588
Steve,-0.603260,-1.056425,-0.772950,-0.860758,0.144331
Wes,-0.607245,NaN,NaN,-1.586719,1.457611
Jim,1.848431,0.183814,-1.303743,1.539212,0.290445
Travis,1.414039,0.994386,-0.827250,-0.499318,0.236633


Now suppose I want to sum together the columns by group.

In [22]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
            'd': 'blue', 'e': 'red', 'f': 'orange'} # key 'f' is unused and okay

by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,-2.268498,-0.274423
Steve,-1.633709,-1.515354
Wes,-1.586719,0.850367
Jim,0.235469,2.322691
Travis,-1.326568,2.645058


In [23]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [24]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 10.1.4 Grouping with Functions

Suppose you wanted to group by length of the names; pass the `len` function.

In [25]:
people.groupby(len).sum()

,a,b,c,d,e
3,1.901719,0.474447,-2.541441,-1.078306,0.522469
5,-0.603260,-1.056425,-0.772950,-0.860758,0.144331
6,1.414039,0.994386,-0.827250,-0.499318,0.236633


Mixing functions with arrays, dicts, or Series is okay -> converted to arrays internally.

In [26]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.607245  0.290632 -1.237698 -1.586719 -1.225588
  two  1.848431  0.183814 -1.303743  1.539212  0.290445
5 one -0.603260 -1.056425 -0.772950 -0.860758  0.144331
6 two  1.414039  0.994386 -0.827250 -0.499318  0.236633

### 10.1.5 Grouping by Index Levels

For hierarchically indexed datasets, you can aggregate using one of the levels of an axis index.

In [27]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      1.117567 -0.664845  1.286682 -0.246313  0.568480
1      0.112494  0.338262 -0.116450  1.076455 -0.342323
2      2.042780  1.671583 -2.392461 -1.917083  1.942340
3      0.641837  1.026628  1.430207 -1.149399  0.693632

To group by level, pass the level number or name using `level`.

In [28]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 Data Aggregation

### 10.2.1 Column-Wise and Mutiple Function Application

### 10.2.2 Returning Aggregated Data Without Row Indexes

## 10.3 Apply: General split-apply-combine

### 10.3.1 Suppressing the Group Keys

### 10.3.2 Quantile and Bucket Analysis

### 10.3.3 Example: Filling Missing Values with Group-Specific Values

### 10.3.4 Example: Random Sampling and Permutation

### 10.3.5 Example: Group Weighted Average and Correlation

### 10.3.6 Example: Group-Wise Linear Regression

## 10.4 Pivot Tables and Cross-Tabulation

### 10.4.1 Cross-Tabulations: Crosstab

## 10.5